In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

import time

In [2]:
country = "CO"
geo2_nq = "CO_municipio"
geo1_nq = "CO_departamento"
geo1_ipums = 'GEO1_CO2005'
geo2_ipums = 'GEO2_CO2005'
year = '2005'

In [3]:
census = pd.read_csv("../Data/"+country+"/sample/"+"ipumsi_00022.csv")

In [4]:
netquest = pd.read_csv("../Data/"+country+"/panel/"+country+"_netquest-panel.csv")

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (6,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
ipums_geo2 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_GEO2_CO2005.csv',encoding='Latin1', names=['code','name'], skiprows=1)
ipums_geo1 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_GEO1_CO2005.csv',encoding='Latin1', names=['code','name'], skiprows=1)

In [37]:
ipums_geodf = census[[geo1_ipums,geo2_ipums]]
ipums_geodf.columns = ['geo1_code','geo2_code']
ipums_geodf.drop_duplicates(inplace=True)

ipums_geodf['geo1_name'] = ipums_geodf.merge(ipums_geo1, 
                                             how='left', 
                                             left_on = "geo1_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

ipums_geodf['geo2_name'] = ipums_geodf.merge(ipums_geo2, 
                                             how='left', 
                                             left_on = "geo2_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

We have to get the melting action going here.

In [38]:
ipums_geodf = ipums_geodf.geo2_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo2_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo2_name')], value_name = "geo2_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo2_name'])
ipums_geodf.geo2_name = ipums_geodf.geo2_name.str.strip()

In [39]:
ipums_geodf = ipums_geodf.geo1_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo1_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo1_name')], value_name = "geo1_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo1_name'])

ipums_geodf.geo1_name = ipums_geodf.geo1_name.str.strip()

Now to Netquest.

In [18]:
# province_labels = pd.read_excel("../Data/CL/provincia_dict.xlsx", encoding='Latin1', header=None)
nq_geodf = netquest[[geo1_nq,geo1_nq+'_name',geo2_nq,geo2_nq+'_name']]
nq_geodf.columns = ['geo1_code','geo1_name','geo2_code','geo2_name']
nq_geodf.drop_duplicates(inplace=True)

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Finding the duplicates in nq_geodf.geo2_code, and keeping the combinations that occur most often (which are quite clearly the good ones).

In [25]:
nq_geodf['count'] = nq_geodf.apply(
    lambda r: sum((netquest[geo1_nq]==r['geo1_code'])&(netquest[geo2_nq]==r['geo2_code'])),
                                   axis=1)
nq_geodf = nq_geodf.sort_values(['geo2_code','count']) \
    [~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]
nq_geodf = nq_geodf[~((nq_geodf.geo2_code.duplicated(keep=False))&(nq_geodf['count']<2))]

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


A fuzzy-join function:

In [27]:
def fuzzy_join(df1, df2, varname, subset=False, cutoff=90):
    '''Returns a dataframe the length of df1, with the matches on the given variable
    
        Assumes the var is named the same in both dataframes.
        
        subset is a tuple or list of length 2 whose first element is a string indicating which
        variable in df1 has to be equal to which variable in df2 (specified by 2nd element)
        '''
    #figure out if there is a constraint
    if not subset:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)
        
    else:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)    

    
    # different match cases
    #morRatioMatch = ratio_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    oneRatioMatch = ratio_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noRatioMatch = ratio_matches.apply(lambda l: (l[0][1]<cutoff))

    #morPartiMatch = parti_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    onePartiMatch = parti_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noPartiMatch = parti_matches.apply(lambda l: (l[0][1]<cutoff))
    
    # pick out what's better
    matches = pd.Series([(np.nan,np.nan,np.nan)]*len(df1),index=df1.index)
    matches.loc[oneRatioMatch] = ratio_matches.loc[oneRatioMatch].apply(lambda l: l[0])
    matches.loc[(noRatioMatch&onePartiMatch)] = parti_matches.loc[(noRatioMatch&onePartiMatch)]\
        .apply(lambda l: l[0])
    matches.loc[(noRatioMatch&noPartiMatch)] = parti_matches.loc[(noRatioMatch&noPartiMatch)]\
        .apply(lambda l: l[0])
    
    df = pd.DataFrame()
    df['name'] = matches.apply(lambda l: l[0])
    df['score'] = matches.apply(lambda l: l[1])
    df['index'] = matches.apply(lambda l: l[2])
    df['parti_matches'] = parti_matches
    df['ratio_matches'] = ratio_matches
    
    return df

Substitutions as found necessary below

In [40]:
ipums_geodf.loc[(ipums_geodf.geo1_name=="Guania"),'geo1_name'] = 'Guainía'
ipums_geodf.loc[(ipums_geodf.geo1_name=="Valle"),'geo1_name'] = 'Valle del Cauca'
ipums_geodf.loc[(ipums_geodf.geo2_name=="Itagui"),'geo2_name'] = 'Itagüí'

nq_geodf.loc[(nq_geodf.geo2_name=="Ubaté"),'geo2_name'] = 'Villa de San Diego de Ubate'
nq_geodf.loc[(nq_geodf.geo2_name=="Machetá"),'geo2_name'] = 'Macheta'
nq_geodf.loc[(nq_geodf.geo2_name=="Tuchin"),'geo2_name'] = 'San Andrés Sotavento'
nq_geodf.loc[(nq_geodf.geo2_name=="Santa Cruz de Mompox"),'geo2_name'] = 'Mompós'
nq_geodf.loc[(nq_geodf.geo2_name=="Ramiquirí"),'geo2_name'] = 'Ramiriquí'
nq_geodf.loc[(nq_geodf.geo2_name=="Bogotá Distrito Capital (D. C.)"),'geo2_name'] = 'Bogotá'
nq_geodf.loc[(nq_geodf.geo2_name=="Anzá"),'geo2_name'] = 'Anza'
nq_geodf.loc[(nq_geodf.geo2_name=="Suán"),'geo2_name'] = 'Suan'
nq_geodf.loc[(nq_geodf.geo2_name=="Villa Gamuez (La Hormiga)"),'geo2_name'] = 'Valle del Guamuez'
nq_geodf.loc[(nq_geodf.geo2_name=="Imúes"),'geo2_name'] = 'Imués'
nq_geodf.loc[(nq_geodf.geo2_name=="San Jose de Ure"),'geo2_name'] = 'Montelíbano'
nq_geodf.loc[((nq_geodf.geo2_name=="Guachené") | (nq_geodf.geo2_name=="Guachene")
             ),'geo2_name'] = 'Caloto'
nq_geodf.loc[(nq_geodf.geo2_name=="Capitanejo"),'geo1_name'] = 'Santander'
nq_geodf.loc[(nq_geodf.geo2_name=="Bojacá"),'geo1_name'] = 'Cundinamarca'
nq_geodf.loc[(nq_geodf.geo2_name=="Beltrán"),'geo1_name'] = 'Cundinamarca'
nq_geodf.loc[(nq_geodf.geo2_name=="Soacha"),'geo1_name'] = 'Cundinamarca'

Using the new tool:

In [41]:
nq_geodf[['geo1_match_name','geo1_match_score','geo1_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo1_name')[['name','score','index']]

In [42]:
nq_geodf[['geo2_match_name','geo2_match_score','geo2_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo2_name', 
             subset=['geo1_match_name','geo1_name']
            )[['name','score','index']]

## De-duping

In [43]:
tmp = nq_geodf.copy()

**What is still an issue??**

In [44]:
tmp[tmp.duplicated('geo2_code', keep=False)].sort_values('geo2_code')\
[["geo2_code","geo1_name","geo1_match_name","geo2_name","geo2_match_name","geo2_match_score","geo2_match_index"]]

,geo2_code,geo1_name,geo1_match_name,geo2_name,geo2_match_name,geo2_match_score,geo2_match_index


In [45]:
tmp[tmp.geo2_match_score<90][['geo1_name',
                                        'geo1_match_name',
                                        'geo2_name',
                                        'geo2_match_name',
                                        'geo2_match_score']
                                      ].sort_values('geo1_name'
                                      )

,geo1_name,geo1_match_name,geo2_name,geo2_match_name,geo2_match_score
17647,Antioquia,Antioquia,Abriaqui,Abriaquí,88
7128,Antioquia,Antioquia,Briseño,Briceño,86
6265,Antioquia,Antioquia,Guatapé,Guatape,86
16450,Antioquia,Antioquia,Sonsón,Sonson,83
10472,Bolivar,Bolívar,Turbana,Turbaná,86
84840,Boyacá,Boyacá,Zetaquirá,Zetaquira,89
88537,Boyacá,Boyacá,Gámeza,Gameza,83
65523,Boyacá,Boyacá,Tunungua,Tununguá,88
106893,Boyacá,Boyacá,Briseño,Briceño,86
142093,Boyacá,Boyacá,Beteitiva,Betéitiva,89


Success! This can be the geo-df.

In [46]:
nq_geodf = tmp

Get a crosswalk for these!

In [47]:
has_ipums_geo = nq_geodf.geo2_match_index.notna()

In [48]:
nq_geodf['IPUMS_geo2_code'] = np.nan

nq_geodf.loc[has_ipums_geo,'IPUMS_geo2_code'] = nq_geodf[has_ipums_geo]\
                            .geo2_match_index\
                            .astype(int)\
                            .apply(
                                lambda i: ipums_geodf.loc[i,'geo2_code']
                            .astype(int)
)

In [49]:
geo2_centroids = pd.read_csv('../Data/'+country+'/geography/'+country+'_geo_centroids.csv', encoding='latin1')

In [50]:
geo2_centroids.head()

,Y,X,CNTRY_NAME,ADMIN_NAME,CNTRY_CODE,IPUM2005,MUNI2005,PARENT
0,13.049822,-81.451803,Colombia,"Providencia, San AndrÃ©s",170,88001,88001,88
1,5.547092,-74.708979,Colombia,La Dorada,170,17001,17380,17
2,5.228689,-75.522443,Colombia,"Aranzazu, Filadelfia, Neira",170,17002,17486,17
3,5.728838,-74.115734,Colombia,"Otanche, Pauna, TununguÃ¡",170,15001,15507,15
4,6.218698,-72.617648,Colombia,"JericÃ³, Sativanorte, El Cocuy, La Uvita, Susa...",170,15002,15244,15


In [51]:
nq_geodf_merged = nq_geodf.merge(geo2_centroids[['ADMIN_NAME','Y','X','IPUM'+year]], 
               left_on='IPUMS_geo2_code',
               right_on="IPUM"+year,
               how='left'
              ).drop('IPUM'+year, axis=1)

In [53]:
panel_geo = netquest.merge(nq_geodf_merged[['X','Y','geo2_code']],
               left_on=geo2_nq,
               right_on='geo2_code',
               how='left'
              ).drop('geo2_code',axis=1)

In [54]:
panel_geo.head()

,Unnamed: 0,gp_codigo,p_sexo,panelistAge,pv_fecha_modificacion,pais,CodigoPostal,CO_departamento,CO_departamento_name,CO_municipio,...,P7_5,P7_6,P7_7,P8,P10,P11,P12,P13,X,Y
0,18,00004f5dafd3bbca,2,31,2018/08,CO,230001,12.0,Córdoba,454.0,...,NaN,NaN,NaN,99.0,NaN,3.0,2.0,2.0,-75.957358,8.586518
1,97,000198cf7584bd84,2,32,2017/08,CO,111211,1.0,Bogotá Distrito Capital (D. C.),1.0,...,NaN,NaN,NaN,3.0,NaN,2.0,3.0,3.0,-74.180799,4.314191
2,98,00019c4339ffabf9,2,32,2018/03,CO,230001,12.0,Córdoba,454.0,...,NaN,NaN,NaN,4.0,NaN,2.0,3.0,3.0,-75.957358,8.586518
3,111,0001c2c501282b06,2,31,2019/01,CO,410017,17.0,Huila,625.0,...,NaN,NaN,NaN,5.0,NaN,1.0,1.0,1.0,-75.309516,3.014506
4,118,0001dab5f80715f3,1,24,2017/08,CO,NaN,1.0,Bogotá Distrito Capital (D. C.),1.0,...,NaN,NaN,NaN,99.0,NaN,3.0,3.0,3.0,-74.180799,4.314191


In [55]:
panel_geo.shape[0]==netquest.shape[0]

True

In [55]:
panel_geo.to_csv('../Data/'+country+'/panel/'+country+'_netquest-panel_geo.csv')

In [56]:
census_geo = census.merge(geo2_centroids[['ADMIN_NAME','X',"Y",'IPUM'+year]],
                          left_on = geo2_ipums,
                          right_on='IPUM'+year,
                          how='left'
                         ).drop('IPUM'+year,axis=1)

In [57]:
census_geo.shape[0]==census.shape[0]

True

In [58]:
census_geo.to_csv('../Data/'+country+'/sample/'+country+'_ipums-census_geo.csv')